In [1]:
from datasets import load_from_disk

import torch
import numpy as np
import random
import gc

from src.model import get_model, get_tokenizer

from src.preprocessing import prepare_dataset
from src.model_operations import get_gradients

[2025-05-14 12:25:32,052] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/lukashinterleitner/anaconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lukashinterleitner/anaconda3/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
torch.cuda.empty_cache()
gc.collect()

84

In [9]:
model_name = "allenai/OLMo-1B-hf"

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")
model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf")

OSError: allenai/OLMo-1B-0724-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [10]:
model = get_model(model_name)
tokenizer = get_tokenizer(model_name)

print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)

Device: cuda:0
Model parameters: 1176764416
OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_e

OSError: allenai/OLMo-1B-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [36]:
use_gpu = True

device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
device

device(type='cuda')

In [9]:
model.eval()

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

In [10]:
dataset = load_from_disk(lima_paraphrased_dataset_path)
dataset

Dataset({
    features: ['id', 'messages', 'paraphrased_messages'],
    num_rows: 988
})

In [11]:
dataset["messages"][0][0]["content"]

'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'

In [12]:
dataset["messages"][0][1]["content"]

'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells migrate over long distances in response to injury (Imitola et al., 2004) and they migrate from specific stem-cell locations (e.g.

In [16]:
original_dataset_config = get_dataset_config(model, sft_messages_key="messages")
original_dataset_config

DatasetConfig(chat_template='tulu', preference_chosen_key='chosen', preference_rejected_key='rejected', sft_messages_key='messages', binary_messages_key='messages', label='binary_labels', convert_preference_to_binary_dataset=False, max_token_length=2048, max_prompt_token_length=None, sanity_check=False, sanity_check_max_samples=100, batched=True, load_from_cache_file=True, num_proc=12, train_only_on_prompt=True, ncols=2)

In [17]:
paraphrased_dataset_config = get_dataset_config(model, sft_messages_key="paraphrased_messages")
paraphrased_dataset_config

DatasetConfig(chat_template='tulu', preference_chosen_key='chosen', preference_rejected_key='rejected', sft_messages_key='paraphrased_messages', binary_messages_key='messages', label='binary_labels', convert_preference_to_binary_dataset=False, max_token_length=2048, max_prompt_token_length=None, sanity_check=False, sanity_check_max_samples=100, batched=True, load_from_cache_file=True, num_proc=12, train_only_on_prompt=True, ncols=2)

In [14]:
sample_size = 5

In [18]:
original_dataset_tokenized = prepare_dataset(dataset=dataset, dataset_config=original_dataset_config, tokenizer=tokenizer, sample_size=sample_size)

In [19]:
paraphrased_dataset_tokenized = prepare_dataset(dataset=dataset, dataset_config=paraphrased_dataset_config, tokenizer=tokenizer, sample_size=sample_size)

In [21]:
gradients_sample_0 = get_gradients(model, original_dataset_tokenized[0], device=device)

In [22]:
for key, value in gradients_sample_0.items():
    print(f"{key}: {value.shape}")

model.embed_tokens.weight: torch.Size([50304, 2048])
model.layers.0.self_attn.q_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.k_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.v_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.o_proj.weight: torch.Size([2048, 2048])
model.layers.0.mlp.gate_proj.weight: torch.Size([8192, 2048])
model.layers.0.mlp.up_proj.weight: torch.Size([8192, 2048])
model.layers.0.mlp.down_proj.weight: torch.Size([2048, 8192])
model.layers.1.self_attn.q_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.k_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.v_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.o_proj.weight: torch.Size([2048, 2048])
model.layers.1.mlp.gate_proj.weight: torch.Size([8192, 2048])
model.layers.1.mlp.up_proj.weight: torch.Size([8192, 2048])
model.layers.1.mlp.down_proj.weight: torch.Size([2048, 8192])
model.layers.2.self_attn.q_proj.weight: torch.Size([2048, 2

In [24]:
original_dataset_tokenized[0]["input_ids"]

tensor([   29,    93,  4537, 49651,   187,  5804,  3998,  1341,  2118,    32,
         2896,  4866,   309,  1599,  1048,  4181, 10346,   313, 11499, 31821,
         1561,   253,  3998,   760,   481,   187,    29,    93,   515,  5567,
        49651,   187,   510,  1953,   310,  4942,  3862,   285,   581,   943,
         1379,   715,  2395,   326,   253,  3998,   417,   760,  8414,   273,
         8512,    13,   533,   671, 42782,  1341,   313, 13821,   422,  1341,
           10,   285,   638,    14,  2225,  3875, 16069,  8424,  1341,    15,
         5488,    13,   347,  4619,  7715,    14, 30202,  1346,   452,  4860,
           13, 16743,  3924,   310,  1077,  1774,    13,   347,   253,  6684,
        24022,  3998,   310,  1077,  1027,   432,   253,  6782,  3998,    15,
          187,  6436,    13,   846,   256, 12545,   949,  2710, 16516,    13,
          253,  3662,   281,   253,  1953,   310,  2686, 24678,  2969,    27,
         6279,    13,  3998,  1341, 31690,    15,   187,   688, 

In [25]:
decoded_input = tokenizer.decode(token_ids=original_dataset_tokenized[0]["input_ids"])
decoded_input

'<|user|>\nCan brain cells move? By movement I mean long distance migration (preferably within the brain only).\n<|assistant|>\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells mi

In [26]:
decoded_output = tokenizer.decode(token_ids=[(tokenizer.pad_token_id if token == -100 else token) for token in original_dataset_tokenized[0]["labels"]])
decoded_output

'<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|>The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of mig

In [29]:
original_dataset_tokenized[0]["input_ids"]

tensor([   29,    93,  4537, 49651,   187,  5804,  3998,  1341,  2118,    32,
         2896,  4866,   309,  1599,  1048,  4181, 10346,   313, 11499, 31821,
         1561,   253,  3998,   760,   481,   187,    29,    93,   515,  5567,
        49651,   187,   510,  1953,   310,  4942,  3862,   285,   581,   943,
         1379,   715,  2395,   326,   253,  3998,   417,   760,  8414,   273,
         8512,    13,   533,   671, 42782,  1341,   313, 13821,   422,  1341,
           10,   285,   638,    14,  2225,  3875, 16069,  8424,  1341,    15,
         5488,    13,   347,  4619,  7715,    14, 30202,  1346,   452,  4860,
           13, 16743,  3924,   310,  1077,  1774,    13,   347,   253,  6684,
        24022,  3998,   310,  1077,  1027,   432,   253,  6782,  3998,    15,
          187,  6436,    13,   846,   256, 12545,   949,  2710, 16516,    13,
          253,  3662,   281,   253,  1953,   310,  2686, 24678,  2969,    27,
         6279,    13,  3998,  1341, 31690,    15,   187,   688, 

In [47]:
output = model.generate(original_dataset_tokenized[0]["input_ids"].reshape(1, -1), max_new_tokens=100)

In [52]:
dataset[0]["messages"][0]["content"]

'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'

In [53]:
inputs = tokenizer(dataset[0]["messages"][0]["content"], return_tensors="pt")

In [57]:
generate_ids = model.generate(inputs.input_ids, max_new_tokens=100, return_full_text=False)

ValueError: The following `model_kwargs` are not used by the model: ['return_full_text'] (note: typos in the generate arguments will also show up in this list)

In [56]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).\nI have a question about the brain.\nI have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have a question about the brain. I have'

In [48]:
print(output)

tensor([[   29,    93,  4537, 49651,   187,  5804,  3998,  1341,  2118,    32,
          2896,  4866,   309,  1599,  1048,  4181, 10346,   313, 11499, 31821,
          1561,   253,  3998,   760,   481,   187,    29,    93,   515,  5567,
         49651,   187,   510,  1953,   310,  4942,  3862,   285,   581,   943,
          1379,   715,  2395,   326,   253,  3998,   417,   760,  8414,   273,
          8512,    13,   533,   671, 42782,  1341,   313, 13821,   422,  1341,
            10,   285,   638,    14,  2225,  3875, 16069,  8424,  1341,    15,
          5488,    13,   347,  4619,  7715,    14, 30202,  1346,   452,  4860,
            13, 16743,  3924,   310,  1077,  1774,    13,   347,   253,  6684,
         24022,  3998,   310,  1077,  1027,   432,   253,  6782,  3998,    15,
           187,  6436,    13,   846,   256, 12545,   949,  2710, 16516,    13,
           253,  3662,   281,   253,  1953,   310,  2686, 24678,  2969,    27,
          6279,    13,  3998,  1341, 31690,    15,  

In [49]:
tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenzation_spaces=False)

['<|user|>\nCan brain cells move? By movement I mean long distance migration (preferably within the brain only).\n<|assistant|>\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells m

In [35]:
dataset[0]

{'id': 'lima_0',
 'messages': [{'role': 'user',
   'content': 'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'},
  {'role': 'assistant',
   'content': 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form